$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\e}{\frac{1}{\sigma^2_e}}
\newcommand{\f}{\frac{1}{\sigma^2_{\alpha}}}
$$

# Algorithm to invert a positive definite matrix

# Rohan L Fernando

# December 6, 2018


- Suppose $\mbf{A}$ is a positive definite variance-covariance matrix

- Can write:
\begin{align*}
    \begin{bmatrix}
    \mbf{A}_{11} & \mbf{A}_{12} \\
    \mbf{A}_{21} & \mbf{A}_{22}
    \end{bmatrix}
    &= \begin{bmatrix}
    \mbf{A}_{11}         & \mbf{A}_{11}\mbf{L} \\
    \mbf{L}'\mbf{A}_{11} & \mbf{L}'\mbf{A}_{11}\mbf{L} + \mbf{R} 
      \end{bmatrix}\\
\end{align*}

-  $\mbf{L} = \mbf{A}_{11}^{-1}\mbf{A}_{12}$

-  $\mbf{R} = \mbf{A}_{22} - \mbf{L}'\mbf{A}_{11}\mbf{L}$

### Numerical demonstration of above identity

#### First construct PD $\mbf{A}$ matrix:

In [1]:
using Distributions, LinearAlgebra
X = rand(Binomial(),10,5)

10×5 Array{Int64,2}:
 1  1  1  1  1
 0  0  0  1  0
 1  0  1  0  1
 1  0  1  1  1
 1  1  0  1  1
 0  1  1  1  0
 1  0  1  1  0
 0  1  1  1  1
 0  0  0  0  1
 1  0  0  0  1

In [2]:
A = X'X

5×5 Array{Int64,2}:
 6  2  4  4  5
 2  4  3  4  3
 4  3  6  5  4
 4  4  5  7  4
 5  3  4  4  7

In [3]:
s1 = 1:2
s2 = 3:5
A11 = A[s1,s1]
A12 = A[s1,s2]
A21 = A[s2,s1]
A22 = A[s2,s2]

3×3 Array{Int64,2}:
 6  5  4
 5  7  4
 4  4  7

Show that $\mbf{A}_{12}$ can be written as $\mbf{A}_{11}\mbf{L}$

In [8]:
A11i = inv(A11)
L = A11i*A12
round.(A11*L - A12,digits=3)

2×3 Array{Float64,2}:
 -0.0  0.0  -0.0
 -0.0  0.0   0.0

In fact, any $2\times 1$ vector $\mbf{b}$ can be written as:

$\mbf{A}\mbf{x} = \mbf{b}$

#### Numerical demonstration

In [32]:
b = randn(2)
x = A11i*b
[A11*x b]

2×2 Array{Float64,2}:
 -1.00064   -1.00064 
  0.558491   0.558491

So, can write 
\begin{align*}
\begin{bmatrix}
    \mbf{A}_{11} & \mbf{A}_{12} 
\end{bmatrix}
&= \mbf{A}_{11}\begin{bmatrix}
                   \mbf{I} & \mbf{L} 
                \end{bmatrix}\\
&= \begin{bmatrix}
   \mbf{A}_{11} & \mbf{A}_{11}\mbf{L} 
                \end{bmatrix},
\end{align*}
which is the second block of $\mbf{A}$.

But, because $\mbf{A}_{21} = \mbf{A}'_{12}$, $\mbf{A}_{21}$ can be written as $\mbf{A}_{21} = \mbf{L}'\mbf{A}_{11}$. 

#### Verifying numerically:

In [34]:
round.(L'A11 - A21)

3×2 Array{Float64,2}:
 -0.0  -0.0
  0.0   0.0
 -0.0   0.0

So, the matrix:

$$ 
\mbf{B} = 
\begin{bmatrix}
   \mbf{I} \\
   \mbf{L}'
\end{bmatrix}
\mbf{A}_{11}\begin{bmatrix}
                   \mbf{I} & \mbf{L} 
                \end{bmatrix}
=
\begin{bmatrix}
    \mbf{A}_{11}         & \mbf{A}_{11}\mbf{L} \\
    \mbf{L}'\mbf{A}_{11} & \mbf{L}'\mbf{A}_{11}\mbf{L}
      \end{bmatrix}
$$

differs from $\mbf{A}$ only in that $\mbf{A}_{22}$ may not be equal to $\mbf{L}'\mbf{A}_{11}\mbf{L}$. Can show that $\mbf{B}$ is not positive definite, but $\mbf{A}$ is positive definite. 

In [64]:
Identity(n) = Matrix(I,n,n)
x = rand(5)
B =  [Identity(2)
      L'          ] *A11 *[Identity(2) L]

5×5 Array{Float64,2}:
 6.0  2.0  4.0  4.0  5.0
 2.0  4.0  3.0  4.0  3.0
 4.0  3.0  3.5  4.0  4.0
 4.0  4.0  4.0  4.8  4.4
 5.0  3.0  4.0  4.4  4.7

In [69]:
y = [L[:,1]
     -1
      0
      0
    ]

5-element Array{Float64,1}:
  0.49999999999999994
  0.49999999999999994
 -1.0                
  0.0                
  0.0                

In [70]:
B*y

5-element Array{Float64,1}:
 -1.1102230246251565e-16
  2.220446049250313e-16 
  0.0                   
  0.0                   
  0.0                   

In [75]:
y = [L[:,2]
       0
      -1
       0
    ]
B*y

5-element Array{Float64,1}:
 0.0                   
 1.1102230246251565e-16
 0.0                   
 2.220446049250313e-16 
 8.881784197001252e-16 

In [71]:
y'B*y

5.551115123125782e-17

Can write $\mbf{A}$ as 

\begin{align*}
   \begin{bmatrix}
    \mbf{A}_{11} & \mbf{A}_{12} \\
    \mbf{A}_{21} & \mbf{A}_{22}
    \end{bmatrix}
    &= \begin{bmatrix}
   \mbf{I} \\
   \mbf{L}'
\end{bmatrix}
\mbf{A}_{11}\begin{bmatrix}
                \mbf{I} & \mbf{L} 
             \end{bmatrix}
+
\begin{bmatrix}
   \mbf{0} \\
   \mbf{I}
\end{bmatrix}
\mbf{R}\begin{bmatrix}
              \mbf{0} & \mbf{I} 
        \end{bmatrix}\\
    &= 
    \begin{bmatrix}
     \mbf{I} & \mbf{0} \\
     \mbf{L}' & \mbf{I}
    \end{bmatrix}
    \begin{bmatrix}
     \mbf{A}_{11} & \mbf{0} \\
     \mbf{0}      & \mbf{R}
    \end{bmatrix}
        \begin{bmatrix}
     \mbf{I} & \mbf{L} \\
     \mbf{0} & \mbf{I}
    \end{bmatrix}\\
    &= \mbf{PVP}'
\end{align*}

So, 
$$
\mbf{A}^{-1} = (\mbf{P}')^{-1}\mbf{V}^{-1}\mbf{P}^{-1}
$$

## Inverse of $\mbf{A}$
\begin{align*}
   \begin{bmatrix}
    \mbf{A}^{11} & \mbf{A}^{12} \\
    \mbf{A}^{21} & \mbf{A}^{22}
    \end{bmatrix}
    &= (\mbf{P}')^{-1}\mbf{V}^{-1}\mbf{P}^{-1}\\
    &= 
    \begin{bmatrix}
     \mbf{I} & -\mbf{L} \\
     \mbf{0} & \mbf{I}
    \end{bmatrix}
    \begin{bmatrix}
     \mbf{A}_{11}^{-1} & \mbf{0} \\
     \mbf{0}      & \mbf{R}^{-1}
    \end{bmatrix}
        \begin{bmatrix}
     \mbf{I} & \mbf{0} \\
     -\mbf{L}' & \mbf{I}
    \end{bmatrix} \\
    &= 
    \begin{bmatrix}
     \mbf{I}\\
     \mbf{0}
    \end{bmatrix}
     \mbf{A}_{11}^{-1}
    \begin{bmatrix}
     \mbf{I} & \mbf{0}
    \end{bmatrix}
    +
    \begin{bmatrix}
     -\mbf{L}\\
     \mbf{I}
    \end{bmatrix}
    \mbf{R}^{-1}
    \begin{bmatrix}
     -\mbf{L}' & \mbf{I}
    \end{bmatrix}   
    \end{align*}
    

\begin{align*}
&=
   \begin{bmatrix}
    \mbf{A}_{11}^{-1} & \mbf{0} \\
    \mbf{0}      & \mbf{0}
    \end{bmatrix}
+
   \begin{bmatrix}
    \mbf{L}\mbf{R}^{-1}\mbf{L}' & -\mbf{L}\mbf{R}^{-1} \\
    -\mbf{R}^{-1}\mbf{L}'       & \mbf{R}^{-1}
    \end{bmatrix}
\end{align*}    

## Inverse Function

In [67]:
function invA(A)
    n = size(A,1)
    if n==1
        return 1/A[1,1]
    else
        s1 = 1:n-1
        s2 = n
        A11 = A[s1,s1]
        A11i = invA(A11)
        A12 = A[s1,s2]
        A22 = A[s2,s2]
        L = A11i*A12
        R = A22 - L'A11*L
        Ri = 1/R[1,1]
        Ai = [A11i .+ (Ri.* L*L') -Ri.*L
              -Ri.*L'             Ri.*1]
        return Ai
   end
end

invA (generic function with 1 method)

## Numerical Example

In [27]:
X = [1 2 3
     3 4 5
     6 7 9]
A = X'X

3×3 Array{Int64,2}:
 46  56   72
 56  69   89
 72  89  115

In [25]:
# Julia inverse function
inv(A)

3×3 Array{Float64,2}:
  3.5   -8.0    4.0
 -8.0   26.5  -15.5
  4.0  -15.5    9.5

In [70]:
# Our inverse function
invA(A)

3×3 Array{Float64,2}:
  3.5   -8.0    4.0
 -8.0   26.5  -15.5
  4.0  -15.5    9.5

## Additive Relationship Matrix: $\mbf{A}$

- $a_{ij}$ is two times the kinship probability ($F_{ij}$) that randomly sampled alleles from $i$ and $j$ are IBD

- Homologous alleles are IBD if they can be traced to the same ancestral allele in a founder

- Thus, two different alleles in founders are not IBD

- Homologous alleles are randomly inherited one from each of the parents $m_i$ and and $f_i$ of $i$

- If $j$ is not a descendant of $i$, the genetic relationship of $j$ with $i$ is entirely through the parents of $j$

Thus,
$$
F_{ij} = \frac{1}{2}(F_{jm_i} + F_{jf_i})
$$

and 

$$
F_{ii} = \frac{1}{2}(1 + F_{m_if_i})
$$

Similarly,
$$
a_{ij} = \frac{1}{2}(a_{jm_i} + a_{jf_i})
$$

and 

$$
a_{ii} = \frac{1}{2}(2 + a_{m_if_i})
$$

## Tabular Method to Compute $\mbf{A}$

- Number individuals such that parents preced offspring

- For founders, enter 1 on diagonal and 0 on off-diagonals

- For non-founder $i$ calculate row elements 1 to $i-1$ as the average of the parental row elements

- Set diagonal element $i$ to $\frac{1}{2}(1 + a_{m_if_i})$

- Fill columns by symmetry.

In matrix notation:

$$
\mbf{A}_i = 
\begin{bmatrix}
    \mbf{A}_{i-1}           & \mbf{A}_{i-1}\mbf{q}_i \\
    \mbf{q}'_i\mbf{A}_{i-1} & \frac{1}{2}(2 + a_{m_if_i})
      \end{bmatrix}
$$

where $mbf{q}_i$ has only two non-zero elements that are equal to 0.5 at positions corresponding to $m_i$ and $f_i$.

So,
$$
\mbf{A}_i^{-1}=
   \begin{bmatrix}
    \mbf{A}_{i-1}^{-1} & \mbf{0} \\
    \mbf{0}      & \mbf{0}
    \end{bmatrix}
+
    \begin{bmatrix}
     -\mbf{q}_i\\
          1
    \end{bmatrix}
    \mbf{a}^{ii}
    \begin{bmatrix}
     -\mbf{q}'_i & 1
    \end{bmatrix}
$$  

where $a^{ii} = (a_{ii} - \mbf{q}'_i\mbf{A}_{i-1}\mbf{q}_i)^{-1}$

## Algorithm

- Set $\mbf{A}^{-1} = \mbf{0}$

- Compute $a^{ii}$ for all animals

- For each animal add to $\mbf{A}^{-1}$

   - $a^{ii}$ to (i,i)
   - -1/2$a^{ii}$ to $(i,f_i)$, $(f_i,i)$,$(i,m_i)$,$(m_i,i)$
   - 1/4$a^{ii}$ to $(f_i,f_i)$, $(f_i,m_i)$,$(m_i,f_i)$,$(m_i,m_i)$

## Numerical Example

Pedigree:
$$
\begin{matrix}
i & f & m\\
1 & 0 & 0\\
2 & 0 & 0\\
3 & 1 & 2\\
4 & 1 & 2\\
5 & 3 & 4
\end{matrix}
$$

In [3]:
A2 = [1 0
      0 1]

2×2 Array{Int64,2}:
 1  0
 0  1

In [4]:
q   = [0.5, 0.5]
amf = A2[1,2]
A3 = [A2    A2*q
      q'A2  1 + 0.5amf   ]

3×3 Array{Float64,2}:
 1.0  0.0  0.5
 0.0  1.0  0.5
 0.5  0.5  1.0

In [5]:
q   = [0.5, 0.5, 0]
amf = A3[1,2]
A4 = [A3    A3*q
      q'A3  1 + 0.5amf   ]

4×4 Array{Float64,2}:
 1.0  0.0  0.5  0.5
 0.0  1.0  0.5  0.5
 0.5  0.5  1.0  0.5
 0.5  0.5  0.5  1.0

In [6]:
q   = [0, 0, 0.5, 0.5]
amf = A4[3,4]
A5 = [A4    A4*q
      q'A4  1 + 0.5amf   ]
A = A5

5×5 Array{Float64,2}:
 1.0  0.0  0.5   0.5   0.5 
 0.0  1.0  0.5   0.5   0.5 
 0.5  0.5  1.0   0.5   0.75
 0.5  0.5  0.5   1.0   0.75
 0.5  0.5  0.75  0.75  1.25

## A Inverse

In [7]:
using LinearAlgebra
A2i = Matrix(1.0I, 2,2)

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [8]:
q = [0.5, 0.5]
a33i = 1/(A[3,3] - q'A2*q)
A3i  = [A2i zero(q)
        zero(q')   0] + [-q; 1]*[-q' 1].*a33i

3×3 Array{Float64,2}:
  1.5   0.5  -1.0
  0.5   1.5  -1.0
 -1.0  -1.0   2.0

In [9]:
inv(A[1:3,1:3])

3×3 Array{Float64,2}:
  1.5   0.5  -1.0
  0.5   1.5  -1.0
 -1.0  -1.0   2.0

In [10]:
q    = [0.5, 0.5, 0]
a44i = 1/(A[4,4] - q'A3*q)
A4i  = [A3i zero(q)
        zero(q')   0] + [-q; 1]*[-q' 1].*a44i

4×4 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0
  1.0   2.0  -1.0  -1.0
 -1.0  -1.0   2.0   0.0
 -1.0  -1.0   0.0   2.0

In [11]:
inv(A[1:4,1:4])

4×4 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0
  1.0   2.0  -1.0  -1.0
 -1.0  -1.0   2.0  -0.0
 -1.0  -1.0   0.0   2.0

In [12]:
q    = [0, 0, 0.5, 0.5]
[-q; 1]*[-q' 1]


5×5 Array{Float64,2}:
  0.0   0.0   0.0    0.0   -0.0
  0.0   0.0   0.0    0.0   -0.0
  0.0   0.0   0.25   0.25  -0.5
  0.0   0.0   0.25   0.25  -0.5
 -0.0  -0.0  -0.5   -0.5    1.0

In [15]:
a55i = 1/(A[5,5] - q'A4*q)
A5i  = [A4i zero(q)
        zero(q')   0] + [-q; 1]*[-q' 1].*a55i

5×5 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0   0.0
  1.0   2.0  -1.0  -1.0   0.0
 -1.0  -1.0   2.5   0.5  -1.0
 -1.0  -1.0   0.5   2.5  -1.0
  0.0   0.0  -1.0  -1.0   2.0

In [14]:
inv(A)

5×5 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0  -0.0
  1.0   2.0  -1.0  -1.0  -0.0
 -1.0  -1.0   2.5   0.5  -1.0
 -1.0  -1.0   0.5   2.5  -1.0
  0.0   0.0  -1.0  -1.0   2.0

In [17]:
; jupyter nbconvert --to slides AInverse.ipynb

[NbConvertApp] Converting notebook AInverse.ipynb to slides
[NbConvertApp] Writing 293812 bytes to AInverse.slides.html
